In [1]:
import pandas as pd

housing_train=pd.read_csv('dataset/train.csv')
initial_shape=housing_train.shape

In [2]:
#Function for A.1. Dropping columns having more than 70% of null or na values
def drop_columns_with_70_per_null(df):
    null_count_featues_dict=df.isnull().sum().to_dict()
    for k,v in null_count_featues_dict.items():
        if v!=0 and v/df.shape[0] > .70:
            df.drop(columns=[k],inplace=True)
    return df
    
    

In [3]:
#A. Handling missing quantitative values

#populating LotFrontage by median as it more robust and less sensitive to outliers, quantative value
housing_train['LotFrontage'].fillna(value=housing_train['LotFrontage'].median(),inplace=True)

#MasVnrArea - filling missing values is dependent on MasVnrType as per dataset (when the value of MasVnrType is None, MasVnrArea is zero)
#But as the both vlues are NA it is better to just delete as its a small number as 8. (Also I see these houses are very old from 1975)
housing_train=housing_train[pd.notnull(housing_train['MasVnrArea'])]

#GarageYrBlt - filling by median
housing_train['GarageYrBlt'].fillna(value=housing_train['GarageYrBlt'].median(),inplace=True)

#final check for quantitative values
print("Missing numerical values in dataframe : {}".format(housing_train.select_dtypes(exclude='object').isnull().sum().sum()))

print(housing_train.shape)
housing_train.head()




Missing numerical values in dataframe : 0
(1452, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:


#A. Handling categorical features

#A.1. Dropping columns having more than 70% of null or na values, as they do not contribute for prediction
housing_train_A1=drop_columns_with_70_per_null(housing_train) #dropped 4 columns
print ('columns dropped after "A.1. Dropping columns having more than 70% of null or na values"  = {}'.format(initial_shape[1]-housing_train_A1.shape[1]))
shape_1=housing_train_A1.shape
print(shape_1)
#housing_train_A1.select_dtypes(include='object').isna().any().tolist()
#housing_train_A1.select_dtypes(include='object').isnull().sum()




columns dropped after "A.1. Dropping columns having more than 70% of null or na values"  = 4
(1452, 77)


In [5]:
#A.2 checking if any feature have more than 100 unique values as it can add to many dimensions if one hot coding is done

max_dimensions=100
for col in housing_train_A1.select_dtypes(include='object').columns:
   print(col,housing_train_A1[col].nunique())
   if housing_train_A1[col].nunique() > max_dimensions:
        print ("dropped : {}".format(col))
        housing_train_A1.drop(columns=[col],inplace=True)
housing_train_A1.shape
#No columns were dropped in this scenario, but this is very critical check otherwise we will end up adding 100's of new columns

MSZoning 5
Street 2
LotShape 4
LandContour 4
Utilities 2
LotConfig 5
LandSlope 3
Neighborhood 25
Condition1 9
Condition2 8
BldgType 5
HouseStyle 8
RoofStyle 6
RoofMatl 8
Exterior1st 15
Exterior2nd 16
MasVnrType 4
ExterQual 4
ExterCond 5
Foundation 6
BsmtQual 4
BsmtCond 4
BsmtExposure 4
BsmtFinType1 6
BsmtFinType2 6
Heating 6
HeatingQC 5
CentralAir 2
Electrical 5
KitchenQual 4
Functional 7
FireplaceQu 5
GarageType 6
GarageFinish 3
GarageQual 5
GarageCond 5
PavedDrive 3
SaleType 9
SaleCondition 6


(1452, 77)

In [6]:
'''
Thinking for this approach but not sure as of now

#B.1 Approach we are trying to use is predicting the missing values using KNN algorithm, as it seems to be the
#most suitable approach rather than just keeping the unkown or deleting some values

from sklearn.neighbors import KNeighborsClassifier
import math

train=housing_train_A1[housing_train_A1['FireplaceQu'].notnull()]
test=housing_train_A1[housing_train_A1['FireplaceQu'].isnull()]
#train_x=train.drop(columns=['id','FireplaceQu'])

if y.shape[0]+x.shape[0] == shape_1[0]:
    print("No missing data")

#Deciding kvalue by sqrt of n and keeping it odd
k= math.floor(math.sqrt(x.shape[0])) if math.floor(math.sqrt(x.shape[0])) % 2 !=0 else math.ceil(math.sqrt(x.shape[0])) 
print("k-value for KNN: {}".format(k))
train_x=train.drop(columns=['Id','FireplaceQu'])
train_y=train['FireplaceQu']
model = KNeighborsClassifier(n_neighbors=k)
model.fit(train_x,train_y)

#knn_impute(target=df['Age'], attributes=df.drop(['Age', 'PassengerId'], 1),
#                                    aggregation_method="median", k_neighbors=10, numeric_distance='euclidean',
#                                    categorical_distance='hamming', missing_neighbors_threshold=0.8)
'''

'\nThinking for this approach but not sure as of now\n\n#B.1 Approach we are trying to use is predicting the missing values using KNN algorithm, as it seems to be the\n#most suitable approach rather than just keeping the unkown or deleting some values\n\nfrom sklearn.neighbors import KNeighborsClassifier\nimport math\n\ntrain=housing_train_A1[housing_train_A1[\'FireplaceQu\'].notnull()]\ntest=housing_train_A1[housing_train_A1[\'FireplaceQu\'].isnull()]\n#train_x=train.drop(columns=[\'id\',\'FireplaceQu\'])\n\nif y.shape[0]+x.shape[0] == shape_1[0]:\n    print("No missing data")\n\n#Deciding kvalue by sqrt of n and keeping it odd\nk= math.floor(math.sqrt(x.shape[0])) if math.floor(math.sqrt(x.shape[0])) % 2 !=0 else math.ceil(math.sqrt(x.shape[0])) \nprint("k-value for KNN: {}".format(k))\ntrain_x=train.drop(columns=[\'Id\',\'FireplaceQu\'])\ntrain_y=train[\'FireplaceQu\']\nmodel = KNeighborsClassifier(n_neighbors=k)\nmodel.fit(train_x,train_y)\n\n#knn_impute(target=df[\'Age\'], attribu

In [7]:
#B Handling missing categorical values
#As of now putting unknown class to all features as U1,U2 ... Un for features F1,F2 ... Fn where Fi are having any missing values
#The idea of populating the missing classes ot labels so that we can proceed with one hot encoding. And can revisit these 
#columns if the preidiction accuracy is good

#columns having null values
housing_train_A1_cat=housing_train_A1.select_dtypes(include='object')
nan_columns=housing_train_A1_cat.columns[housing_train_A1_cat.isna().any()].tolist()

for iter,cat in enumerate(nan_columns):
    unknown_class_name="U"+str(iter)
    print (iter,cat,unknown_class_name)
    housing_train_A1[cat].fillna(value=unknown_class_name,inplace=True)

housing_train_A1.select_dtypes(include='object').isnull().sum()

0 BsmtQual U0
1 BsmtCond U1
2 BsmtExposure U2
3 BsmtFinType1 U3
4 BsmtFinType2 U4
5 Electrical U5
6 FireplaceQu U6
7 GarageType U7
8 GarageFinish U8
9 GarageQual U9
10 GarageCond U10


MSZoning         0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
KitchenQual      0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
SaleType         0
SaleCondition    0
dtype: int64

In [8]:
'''

if feature_null.shape[0]+feature_notnull.shape[0] == shape_1[0]:
    print("No missing data")
enc.fit(feature_notnull[['FireplaceQu']])
enc.categories_
sparse_out=enc.transform(feature_notnull[['FireplaceQu']])
print(sparse_out)

feature_notnull.head()

one_category_df=housing_train_A1.drop(columns=housing_train_A1.select_dtypes(include='object').drop(columns=['FireplaceQu']).columns)
feature_notnull=one_category_df[housing_train_A1.FireplaceQu.notnull()]
feature_null=one_category_df[housing_train_A1.FireplaceQu.isnull()]
if feature_null.shape[0]+feature_notnull.shape[0] == shape_1[0]:
    print("No missing data")
enc.fit(feature_notnull)
enc.categories_
sparse_out=enc.transform(feature_notnull)
print(sparse_out)
'''

'\n\nif feature_null.shape[0]+feature_notnull.shape[0] == shape_1[0]:\n    print("No missing data")\nenc.fit(feature_notnull[[\'FireplaceQu\']])\nenc.categories_\nsparse_out=enc.transform(feature_notnull[[\'FireplaceQu\']])\nprint(sparse_out)\n\nfeature_notnull.head()\n\none_category_df=housing_train_A1.drop(columns=housing_train_A1.select_dtypes(include=\'object\').drop(columns=[\'FireplaceQu\']).columns)\nfeature_notnull=one_category_df[housing_train_A1.FireplaceQu.notnull()]\nfeature_null=one_category_df[housing_train_A1.FireplaceQu.isnull()]\nif feature_null.shape[0]+feature_notnull.shape[0] == shape_1[0]:\n    print("No missing data")\nenc.fit(feature_notnull)\nenc.categories_\nsparse_out=enc.transform(feature_notnull)\nprint(sparse_out)\n'

In [9]:
#C. OneHotEncoder to bring categorical values to numerical scale

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
'''

print(cat_col)

housing_train_A1[cat_col] = housing_train_A1[cat_col].apply(lambda col: enc.fit_transform(col))
'''

df_transformed=enc.fit_transform(housing_train_A1.select_dtypes(include='object')).toarray()

#enc.categories_
#transformed_array=enc.transform(housing_train_A1).toarray()
#inverse_array=enc.inverse_transform(transformed_array)
#enc.get_feature_names()
print("transformed df : {}" .format(df_transformed.shape))
print(type(df_transformed))
new_housing_train=pd.DataFrame(data=df_transformed)
print(type(new_housing_train))
print(new_housing_train.shape)
new_housing_train.head()


full_housing_train=pd.concat([housing_train_A1,new_housing_train],axis=1,join="inner")
print("after concat : {}" .format(full_housing_train.shape))
#generating categorical column list
cat_features= housing_train_A1.dtypes==object
cat_col=[]
for colu in housing_train_A1.select_dtypes(include='object').columns:
    cat_col.append(colu)

#deleting old original columns
full_housing_train.drop(columns=cat_col,inplace=True)

#deleting id column as it does not have any significance
full_housing_train.drop(columns='Id',inplace=True)

print(full_housing_train.shape)
full_housing_train



transformed df : (1452, 250)
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
(1452, 250)
after concat : (1444, 327)
(1444, 287)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,240,241,242,243,244,245,246,247,248,249
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,50,85.0,14115,5,5,1993,1995,0.0,732,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,20,75.0,10084,8,5,2004,2005,186.0,1369,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,60,69.0,10382,7,6,1973,1973,240.0,859,32,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,50,51.0,6120,7,5,1931,1950,0.0,0,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
9,190,50.0,7420,5,6,1939,1950,0.0,851,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
'''techniques for dimension reduction
1. check corr and select one of the two features if correlation is high
2. check p-value, with alternate hypothesis as there is relation between the given two attributes
3. PCA
'''
import seaborn as sns
import numpy as np

cor=full_housing_train.corr()
#sns.heatmap(cor)
cor_abs=cor.abs()
print(cor_abs.shape)
columns = np.full((cor.shape[0],), True, dtype=bool)
#cor_abs
columns

(287, 287)


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [11]:

for i in range(cor_abs.shape[0]):
    for j in range(i+1,cor_abs[0].shape[0]):
        if cor_abs.iloc[i][j] > 0.7:
            #print(cor_abs[i][j])
            if columns[j]:
                columns[j]=False
true_columns=full_housing_train.columns[columns]
full_housing_train=full_housing_train[true_columns]
full_housing_train.shape
#Correlation reduced 12 features

(1444, 275)

In [12]:
full_housing_train.to_csv('after_correlation.csv')